In [1]:
reload_lamb()

# Tutorial: adding composition operations
### Author: Kyle Rawlins

You can write a new composition rule in three ways, in order of complexity (from simplest to hardest):

1. Write a metalanguage combinator that describes the rule and instantiate it as a composition rule.  This involves variants of the following syntax:
        system.add_binary_rule(combinator, "NAME")
2. Write a python function that describes the rule (as an operation over meta-language elements) and instantiate it as a composition rule.  The above `add_rule` call expects curried functions, and python functions are usually uncurried, but the call is essentially the same:
       system.add_binary_rule_uncurried(function, "names")
3. Write a python function that operates on object language elements (instances of `lang.SingletonComposable`) and produces an object language element (typically an instance of `lang.Composite`).

Where possible, you should apply strategy 1, and then 2.  Both of these handle a whole bunch of bookkeeping for you.  Doing 3 correctly in the general case often requires some detailed knowledge of the internals of the lambda notebook, and it is easy to make mistakes with the most obvious implementations.  (Sometimes it is unavoidable, as in the monster example below, or Predicate Abstraction.)

### Example 1.  A unary operator: existential closure

The simplest kind of operation to add is a unary operation, and existential closure is a good example of this.  (See the type shifting tutorial notebook for more examples of unary operations.)  The idea of existential closure is to existentially bind of some variable that is currently exposed as a lambda term in a formula.  The idea is easiest to see in the form of a combinator.  Existential closure is usually introduced at other types, but I will just use `e` for exemplification:

In [2]:
ec_combinator = te("lambda f_<e,t> : Exists x_e : f(x)")
ec_combinator

(λ f_<e,t>: (Exists x_e: f_<e,t>(x_e)))

Any metalanguage function is also a python function, so you can do stuff like the following directly to see how this works:

In [3]:
ec_combinator(lang.cat.content).reduce_all()

(Exists x_e: Cat_<e,t>(x_e))

Adding this to the composition system is straightforward.  The following code makes a copy of one of the default systems and adds the existential closure rule as a unary rule based on the combinator:

In [4]:
system = lang.td_system.copy()
system.add_unary_rule(ec_combinator, "EC")
lang.set_system(system)
system

Composition system: H&K simple (copy)

In [5]:
lang.cat * lang.gray

CompositionResult(results=[⟦[cat gray]⟧ = (λ x_e: (Cat_<e,t>(x_e) & Gray_<e,t>(x_e)))], failures=[⟦[cat gray]⟧ = Type mismatch: '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> and '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> conflict (mode: Function Application), ⟦[gray cat]⟧ = Type mismatch: '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> and '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> conflict (mode: Function Application), ⟦[cat gray]⟧ = Type mismatch: '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> and '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Abstraction), ⟦[gray cat]⟧ = Type mismatch: '⟦gray⟧ = (λ x_e: Gray_<e,t>(x_e))'/<e,t> and '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Abstraction)])

One way to force a unary rule to apply is to use `* None`:

In [6]:
ectest = (lang.cat * lang.gray) * None
ectest

CompositionResult(results=[⟦[[cat gray]]⟧ = (Exists x_e: (Cat_<e,t>(x_e) & Gray_<e,t>(x_e)))], failures=[])

In [7]:
ectest.tree(derivations=True)

,"$[\![\mathbf{\text{[cat gray]}}]\!]^{}_{\left\langle{}e,t\right\rangle{}}$"
$=$,"${[\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \lambda{} g_{\left\langle{}e,t\right\rangle{}} \: . \: \lambda{} x_{e} \: . \: ({f}({x}) \wedge{} {g}({x}))]}(\lambda{} x_{e} \: . \: {Cat}({x}))(\lambda{} x_{e} \: . \: {Gray}({x}))$"
$=$,"${[\lambda{} g_{\left\langle{}e,t\right\rangle{}} \: . \: \lambda{} x_{e} \: . \: ({Cat}({x}) \wedge{} {g}({x}))]}(\lambda{} x_{e} \: . \: {Gray}({x}))$"
$=$,$\lambda{} x_{e} \: . \: ({Cat}({x}) \wedge{} {[\lambda{} x_{e} \: . \: {Gray}({x})]}({x}))$
$=$,$\lambda{} x_{e} \: . \: ({Cat}({x}) \wedge{} {Gray}({x}))$
,$[\![\mathbf{\text{[[cat gray]]}}]\!]^{}_{t}$
$=$,"${[\lambda{} f_{\left\langle{}e,t\right\rangle{}} \: . \: \exists{} x_{e} \: . \: {f}({x})]}(\lambda{} x_{e} \: . \: ({Cat}({x}) \wedge{} {Gray}({x})))$"
$=$,$\exists{} x_{e} \: . \: {[\lambda{} x_{e} \: . \: ({Cat}({x}) \wedge{} {Gray}({x}))]}({x})$
$=$,$\exists{} x_{e} \: . \: ({Cat}({x}) \wedge{} {Gray}({x}))$


In [8]:
ectest.trace()

Full composition trace.  1 path:<br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 1: $[\![\mathbf{\text{cat}}]\!]^{}_{\left\langle{}e,t\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Cat}({x})$<br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 2: $[\![\mathbf{\text{gray}}]\!]^{}_{\left\langle{}e,t\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Gray}({x})$<br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 3: $[\![\mathbf{\text{cat}}]\!]^{}_{\left\langle{}e,t\right\rangle{}}$ * $[\![\mathbf{\text{gray}}]\!]^{}_{\left\langle{}e,t\right\rangle{}}$ leads to: $[\![\mathbf{\text{[cat gray]}}]\!]^{}_{\left\langle{}e,t\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: ({Cat}({x}) \wedge{} {Gray}({x}))$ <b>[by PM]</b><br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 4: $[\![\mathbf{\text{[cat gray]}}]\!]^{}_{\left\langle{}e,t\right\rangle{}}$ leads to: $[\![\mathbf{\text{[[cat gray]]}}]\!]^{}_{t} \:=\: $$\exists{} x_{e} \: . \: ({Cat}({x}) \wedge{} {Gray}({x}))$ <b>[by EC]</b><br />

### Appendix 1: other ways of introducing existential closure

Behind the scenes, `add_unary_rule` constructs a CompositionOp using a factory function.  We could unpack this as follows:

In [9]:
ec_rule = lang.unary_factory(ec_combinator, "EC", typeshift=False)
system = lang.td_system.copy()
system.add_rule(ec_rule)
lang.set_system(system)
system

Composition system: H&K simple (copy)

In [10]:
lang.cat * None

CompositionResult(results=[⟦[cat]⟧ = (Exists x_e: Cat_<e,t>(x_e))], failures=[])

A more elaborate thing would be to try strategy 3 above: write a function that manipulates object language elements to perform existential closure.  There is a bunch of extra complexity that you need to keep track of when doing this, including metalanguage assignments.  The following is an example function that implements existential closure still using the combinator internally; one could of course build the result in some other way (but this is surprisingly easy to get wrong).  Note that the type checking isn't strictly necessary, as it would happen from the combinator -- it is here for example purposes.

In [11]:
def ec_fun_direct(f, assignment=None):
    ts = meta.get_type_system() # load the current type system
    if not (ts.eq_check(f.type, types.type_property)): # is the type a property type?  (<e,t>)
        raise types.TypeMismatch(f, None, "Existential closure for <e,t>")  # if not, raise an exception.
    ec_expr = te("lambda f_<e,t> : Exists x_e : f(x)") # use the combinator
    result = ec_expr.apply(f.content.under_assignment(assignment)).reduce() # reduce the outermost lambda
    return lang.UnaryComposite(f, result, source="EC-direct(%s)" % (f.name))

ec_fun_direct(lang.cat).content

(Exists x_e: ((λ x_e: Cat_<e,t>(x_e)))(x_e))

To use this you need to build a CompositionOp out of it.  Once you have this, add it to the composition system as usual

In [12]:
system = lang.td_system.copy()
system.add_rule(lang.UnaryCompositionOp("EC-direct", ec_fun_direct, typeshift=False, reduce=True))
lang.set_system(system)
system

Composition system: H&K simple (copy)

In [13]:
lang.cat * None

CompositionResult(results=[⟦EC-direct(cat)⟧ = (Exists x_e: Cat_<e,t>(x_e))], failures=[])

### Example 2.  A binary rule: Predicate Modification

What about binary composition operations?  Predicate modification comes built in, but it is useful to see how one might construct PM.  Like the built-in version, this is restricted to type `<e,t>`.

In [14]:
pm_op = %te L f_<e,t> : L g_<e,t> : L x_e : f(x) & g(x)
pm_op

(λ f_<e,t>: (λ g_<e,t>: (λ x_e: (f_<e,t>(x_e) & g_<e,t>(x_e)))))

In [15]:
pm_op(lang.cat.content)(lang.gray.content).reduce_all().derivation

1. ((λ f_<e,t>: (λ g_<e,t>: (λ x_e: (f_<e,t>(x_e) & g_<e,t>(x_e))))))((λ x_e: Cat_<e,t>(x_e)))((λ x_e: Gray_<e,t>(x_e)))
 2. ((λ g_<e,t>: (λ x_e: (Cat_<e,t>(x_e) & g_<e,t>(x_e)))))((λ x_e: Gray_<e,t>(x_e)))    (Recursive reduction of operand 0)
 3. (λ x_e: (Cat_<e,t>(x_e) & ((λ x_e: Gray_<e,t>(x_e)))(x_e)))    (Reduction)
 4. (λ x_e: (Cat_<e,t>(x_e) & Gray_<e,t>(x_e)))    (Recursive reduction of body)

Now, Predicate Modification can be constructed directly using this combinator.

In [16]:
system = lang.td_system.copy()
system.remove_rule("PM")
system.add_binary_rule(pm_op, "PM2")
lang.set_system(system)
system

Composition system: H&K simple (copy)

In [17]:
(lang.cat * lang.gray).tree()

2 composition paths:<br />
Path [0]:<br />
<div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{cat}}]\!]^{}_{\left\langle{}e,t\right\rangle{}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\lambda{} x_{e} \: . \: {Cat}({x})$</span></div></div></div></div></div><div style="align: center; vertical-align: middle; display: table-cell;"><span style="padding:1em;">*</span></div><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{gray}}]\!]^{}_{\left\langle{}e,t\right\rangle{}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\lambda{} x_{e} \: . \: {Gray}({x})$</span></div></div></div></div></div></div></div><div style="display:table-cell;vertical-align:middle;border-left:1px solid #848482;padding:0.5em"><div style="white-space:nowrap; color:blue;"><b>[<span>PM2</span>]</b></div></div></div><div align="center" style="display:table-row;padding: 5px;"><div align="center" style="display:table-cell;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{[cat gray]}}]\!]^{}_{\left\langle{}e,t\right\rangle{}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\lambda{} x_{e} \: . \: ({Cat}({x}) \wedge{} {Gray}({x}))$</span></div></div></div></div></div><div style="display:table-cell;"></div></div></div><br /><br />Path [1]:<br />
<div align="center" style="display:table; margin:5px; border-collapse: collapse;border: 1px solid #848482;"><div align="center" style="display:table-row;border-bottom:1px solid #848482"><div style="display:table-cell;vertical-align:middle;"><div style="display: table;"><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{gray}}]\!]^{}_{\left\langle{}e,t\right\rangle{}}$</span></div></div><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$\lambda{} x_{e} \: . \: {Gray}({x})$</span></div></div></div></div></div><div style="align: center; vertical-align: middle; display: table-cell;"><span style="padding:1em;">*</span></div><div style="display:table-cell;vertical-align:middle;padding: 5px;"><div align="center" style="display:inline-block;padding: 5px;"><div align="center" style="display:table;table-layout:auto;"><div style="display:table-row;"><div align="center" style="display:table-cell;padding-right:2px; padding-left:2px;"><span>$[\![\mathbf{\text{cat}}]\!]^{}_{\left\langle{}e,t\right\rangle{}}$</span></div></div><div style="display:table-row;"><div align="center" style=

See the neo-Davidsonian event semantics fragment for an example of how to write a generalized PM that works for polymorphic types.

### Appendix 2: an object-language implementation for Predicate Modification

Once again, you could write a python function that does the work.  This one still uses the combinator internally.
  * Tangent: your instinct may be to construct the result directly by building up the right TypedExpression.  This is certainly possible, but it is surprisingly tricky to get right; I encourage you to find solutions that involve combinators.
  * One reason is that you would have to deal with some issues in variable renaming to handle the general case of this; using beta reduction via a combinator ensures that this is all taken care of.

In [18]:
pm_op = lang.te("L f_<e,t> : L g_<e,t> : L x_e : f(x) & g(x)")

def pm_fun(fun1, fun2, assignment=None):
    """H&K predicate modification -- restricted to type <e,t>."""
    ts = meta.get_type_system()
    if not (ts.eq_check(fun1.type, types.type_property) and 
            ts.eq_check(fun2.type, types.type_property)):
        raise TypeMismatch(fun1, fun2, "Predicate Modification")
    c1 = fun1.content.under_assignment(assignment)
    c2 = fun2.content.under_assignment(assignment)
    result = pm_op.apply(c1).apply(c2).reduce_all()
    return lang.BinaryComposite(fun1, fun2, result)

### Example 3: Binding evaluation parameters

A second kind of unary operation involves abstracting over a free variable in the metalanguage expression.  This can be thought of as "monstrous" shifting in the sense of Kaplan.  The following code sketches an implementation of this.

In [19]:
reload_lamb()

In [20]:
system = lang.td_system.copy()
speaker = lang.te("speaker_e")
system.assign_controller = lang.AssignmentController(specials=[speaker])
lang.set_system(system)
# note that the predefined examples in lang are defined with a different composition system that has no controller

In [21]:
i = lang.Item("I", lang.te("speaker_e"))
i

⟦I⟧[speaker_e, type e, g] = speaker_e

The following function binds instances of the variable `speaker` to a lambda term.  Notice that this function doesn't do anything to ensure that there are no free instances of `x` in `f`; this is one of the complexities you may have to deal with when implementing non-combinator-based composition operations.  However, this is basically a reasonable assumption for a case like this under normal practice.

In [22]:
def monster_fun(f, assignment=None):
    new_a = lang.Assignment(assignment)
    new_a.update({"speaker": lang.te("x_e")})
    result = meta.LFun(types.type_e, f.under_assignment(new_a), varname="x")
    return lang.UnaryComposite(f, result, source="M(%s)" % (f.name))

monster_fun(i)

⟦M(I)⟧[speaker_e, type e, g] = (λ x_e: x_e)

In [23]:
system.add_rule(lang.UnaryCompositionOp("Monster", monster_fun))

In [24]:
m_test = i * None
m_test

CompositionResult(results=[⟦M(I)⟧[speaker_e, type e, g] = (λ x_e: x_e)], failures=[])

In [25]:
lang.cat * i

CompositionResult(results=[⟦[cat I]⟧[speaker_e, type e, g] = Cat_<e,t>(speaker_e)], failures=[⟦[I cat]⟧[speaker_e, type e, g] = Type mismatch: '⟦I⟧[speaker_e, type e, g] = speaker_e'/e and '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> conflict (mode: Function Application), ⟦[cat I]⟧[speaker_e, type e, g] = Type mismatch: '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> and '⟦I⟧[speaker_e, type e, g] = speaker_e'/e conflict (mode: Predicate Modification), ⟦[cat I]⟧[speaker_e, type e, g] = Type mismatch: '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> and '⟦I⟧[speaker_e, type e, g] = speaker_e'/e conflict (mode: Predicate Abstraction), ⟦[I cat]⟧[speaker_e, type e, g] = Type mismatch: '⟦I⟧[speaker_e, type e, g] = speaker_e'/e and '⟦cat⟧ = (λ x_e: Cat_<e,t>(x_e))'/<e,t> conflict (mode: Predicate Abstraction)])

In [26]:
((lang.cat * i) * None).trace()

Full composition trace.  1 path:<br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 1: $[\![\mathbf{\text{cat}}]\!]^{}_{\left\langle{}e,t\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Cat}({x})$<br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 2: $[\![\mathbf{\text{I}}]\!]^{{speaker}_{e}, g}_{e} \:=\: $${speaker}_{e}$<br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 3: $[\![\mathbf{\text{cat}}]\!]^{}_{\left\langle{}e,t\right\rangle{}}$ * $[\![\mathbf{\text{I}}]\!]^{{speaker}_{e}, g}_{e}$ leads to: $[\![\mathbf{\text{[cat I]}}]\!]^{{speaker}_{e}, g}_{t} \:=\: $${Cat}({speaker}_{e})$ <b>[by FA]</b><br />
&nbsp;&nbsp;&nbsp;&nbsp;Step 4: $[\![\mathbf{\text{[cat I]}}]\!]^{{speaker}_{e}, g}_{t}$ leads to: $[\![\mathbf{\text{M([cat I])}}]\!]^{{speaker}_{e}, g}_{\left\langle{}e,t\right\rangle{}} \:=\: $$\lambda{} x_{e} \: . \: {Cat}({x})$ <b>[by Monster]</b><br />

In [27]:
%lamb ||john|| = John_e
(john * lang.cat) * None # results in vacuous binding

CompositionResult(results=[⟦M([cat john])⟧[speaker_e, type e, g] = (λ x_e: Cat_<e,t>(John_e))], failures=[])